#### Импорт библиотек

In [1]:
import os
import cv2
import fitz
import json
import shutil
import base64
import numpy as np
import requests
import pandas as pd
from Levenshtein import distance as levenshtein_distance
from boxdetect import config
from boxdetect.pipelines import get_checkboxes

from config import (
    TELEGRAM_TOKEN,
    folder_id,
    oauth_token
)

VISION_URL = 'https://ocr.api.cloud.yandex.net/ocr/v1/recognizeText'

#### Создание ключа для Yandex OCR

In [2]:
# Создаем ключ API, обращаемся к папке
def create_token(oauth_token):
    params = {'yandexPassportOauthToken': oauth_token}
    response = requests.post('https://iam.api.cloud.yandex.net/iam/v1/tokens', params=params)
    decode_response = response.content.decode('UTF-8')
    text = json.loads(decode_response)
    iam_token = text.get('iamToken')

    return iam_token

# IAM-токен
IAM_TOKEN = create_token(oauth_token)
# Идентификатор каталога
folder_id = folder_id

#### Функции разбора PDF на страницы и кодировка в base64

In [3]:
# Разбираем PDF файл на страницы и переводим его в png, работает также и с изображениями
def file_to_png(file_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    # Открываем PDF файл
    pdf_document = fitz.open(file_path)

    # Проходим по всем страницам PDF и сохраняем их в формате PNG
    for page_number in range(len(pdf_document)):
        page = pdf_document.load_page(page_number)
        image = page.get_pixmap()

        
        image.save(os.path.join(output_folder, f'img{page_number+1}.png'))


    # Закрываем PDF файл
    pdf_document.close()
    print('Done!')

# Кодировка файла в base 64
def encode_file(file):
  with open(file, 'rb') as file:
    file_content = file.read()
    return base64.b64encode(file_content).decode('UTF-8')


In [4]:
# Функция очистки папки
def clear_folder(folder_path):
    # Проверка существования папки
    if os.path.exists(folder_path):
        # Перебор всех файлов в папке
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            # Удаление файла
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
    else:
        print(f'Папка {folder_path} не найдена')




#### Функция вырезки изображения по красной рамке

In [6]:
# Вырезка рабочая по красной рамке
def extract_inner_red_rectangles(image_path, output_folder):
    # Загрузка изображения
    image = cv2.imread(image_path)
    
    # Конвертация изображения в цветовое пространство HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Определение диапазона красного цвета
    lower_red = np.array([0, 120, 70])
    upper_red = np.array([10, 255, 255])
    mask1 = cv2.inRange(hsv, lower_red, upper_red)
    
    lower_red = np.array([170, 120, 70])
    upper_red = np.array([180, 255, 255])
    mask2 = cv2.inRange(hsv, lower_red, upper_red)
    
    # Объединение масок для получения полного диапазона красного цвета
    mask = mask1 + mask2
    
    # Нахождение контуров на маске
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Создание папки, если она не существует
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Переменная для подсчета сохраненных фрагментов
    saved_count = 0
    
    # Перебор всех найденных контуров
    for contour in contours:
        # Проверка площади контура
        if cv2.contourArea(contour) > 5000:
            # Создание маски для вырезания внутренней области контура
            mask = np.zeros_like(image)
            cv2.drawContours(mask, [contour], -1, (255, 255, 255), thickness=cv2.FILLED)
            
            # Применение маски к исходному изображению
            cut_image = cv2.bitwise_and(image, mask)
            
            # Получение координат прямоугольника вокруг контура
            x, y, w, h = cv2.boundingRect(contour)
            
            # Вырезание области внутри прямоугольника
            cut_image = cut_image[y:y+h, x:x+w]
            
            # Формирование уникального имени файла для каждого фрагмента
            output_path = os.path.join(output_folder, f'crop_{os.path.splitext(os.path.basename(image_path))[0]}_{saved_count}.png')
            
            # Сохранение вырезанного фрагмента в файл
            cv2.imwrite(output_path, cut_image)
                        
            # Увеличение счетчика сохраненных фрагментов
            saved_count += 1

#### Функции предобработки изображений

In [7]:
# Функция бинаризации
def binarize_and_save_image(image_path, output_folder):
    # Загружаем изображение
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Бинаризуем изображение
    _, binary_image = cv2.threshold(image, 120, 255, cv2.THRESH_BINARY)
    
    # Создаем папку, если она не существует
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Сохраняем бинаризированное изображение
    output_path = os.path.join(output_folder, f'bin_{os.path.splitext(os.path.basename(image_path))[0]}.png')
    cv2.imwrite(output_path, binary_image)

In [8]:
# Функция поиска контуров на рамках любого цвета
def find_large_fragments_draw_save(image_path, output_folder):
    # Загружаем изображение
    image = cv2.imread(image_path)
    if image is None:
        print("Не удалось загрузить изображение.")
        return
    
    # Конвертируем в оттенки серого
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Бинаризуем изображение
    _, binary_image = cv2.threshold(gray_image, 120, 255, cv2.THRESH_BINARY_INV)
    
    # Находим контуры
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Фильтруем контуры по площади
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 10000:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # Создаем папку, если она не существует
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Сохраняем изображение с нарисованными границами
    output_path = os.path.join(output_folder, f'contours_{os.path.basename(image_path)}')
    cv2.imwrite(output_path, image)
    print(f"Изображение с нарисованными границами сохранено в {output_folder}")


In [9]:
# Функция для нормализации яркости
def normalize_brightness(image_path, output_folder):
    
    # Создаем папку, если она не существует
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    # Загрузка изображения
    image = cv2.imread(image_path)
    
    # Конвертация в цветовое пространство HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Выделение канала яркости
    value_channel = hsv[:, :, 2]
    
    # Нахождение самой яркой точки
    max_brightness = np.max(value_channel)
    
    # Вычисление коэффициента для нормализации яркости
    scale_factor = 255 / max_brightness if max_brightness > 0 else 1
    
    # Нормализация канала яркости
    value_channel = np.clip(value_channel * scale_factor, 0, 255).astype(np.uint8)
    
    # Обновление канала яркости в изображении HSV
    hsv[:, :, 2] = value_channel
    
    # Конвертация обратно в BGR
    normalized_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    
    # Сохранение изображения
    output_path = os.path.join(output_folder, f'norm_brig{os.path.basename(image_path)}')
    cv2.imwrite(output_path, normalized_image)
    

In [10]:
# Функция для балансировки белого
def balance_white(image_path, output_folder):
    
    # Создаем папку, если она не существует
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    # Загрузка изображения
    image = cv2.imread(image_path)
    
    # Конвертация изображения в пространство LAB
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    
    # Разделение каналов
    l, a, b = cv2.split(lab)
    
    # Нормализация канала L
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    
    # Объединение каналов обратно
    limg = cv2.merge((cl, a, b))
    
    # Конвертация изображения обратно в BGR
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    
    # Сохранение изображения
    output_path = os.path.join(output_folder, f'bal_white{os.path.basename(image_path)}')
    cv2.imwrite(output_path, final)
    


#### Функция предобработки изображения вырезки фрагментов
Здесь нужно попробовать перевести изображение в ЧБ затем подать на определение контуров

In [11]:
def pre_process(image_path, output_folder):
    # Создание папки, если она не существует
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    image = cv2.imread(image_path)
    # ВЫРАВНИВАЕМ БАЛАНС БЕЛОГО
    # Конвертация изображения в пространство LAB
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Разделение каналов
    l, a, b = cv2.split(lab)

    # Нормализация канала L
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)

    # Объединение каналов обратно
    limg = cv2.merge((cl, a, b))

    # Конвертация изображения обратно в BGR
    balanced_white = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    # КОРРЕКТИРУЕМ ЯРКОСТЬ ПО САМОЙ ЯРКОЙ ТОЧКЕ
    # Конвертация в цветовое пространство HSV
    hsv = cv2.cvtColor(balanced_white, cv2.COLOR_BGR2HSV)

    # Выделение канала яркости
    value_channel = hsv[:, :, 2]
    
    # Нахождение самой яркой точки
    max_brightness = np.max(value_channel)
    
    # Вычисление коэффициента для нормализации яркости
    scale_factor = 255 / max_brightness if max_brightness > 0 else 1
    
    # Нормализация канала яркости
    value_channel = np.clip(value_channel * scale_factor, 0, 255).astype(np.uint8)
    
    # Обновление канала яркости в изображении HSV
    hsv[:, :, 2] = value_channel
    
    # Конвертация обратно в BGR
    normalized_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    # НАХОДИМ ГРАНИЦЫ АНКЕТ И ВЫРЕЗАЕМ ПО НИМ ОТДЕЛЬНЫЕ ФРАГМЕНТЫ
    # Конвертируем в оттенки серого
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Бинаризуем изображение
    _, binary_image = cv2.threshold(gray_image, 120, 255, cv2.THRESH_BINARY_INV)
    
    # Находим контуры
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    count = 0

    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        sqr = w*h
        if sqr > 80000:        
            cont = cv2.contourArea(contour)
        

            # Вырезаем фрагмент по контуру
            crop_image = normalized_image[y:y+h, x:x+w]

            # Формирование уникального имени файла для каждого фрагмента
            output_path = os.path.join(output_folder, f'crop_{os.path.splitext(os.path.basename(image_path))[0]}_{count}.png')
            
            # Сохранение вырезанного фрагмента в файл
            cv2.imwrite(output_path, crop_image)
                        
            # Увеличение счетчика сохраненных фрагментов
            count += 1
            print (f'площадь {sqr} __ area {cont}')

#### Функции поворота изображения

In [5]:
def average_slope(lines):
    slopes = []
    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = np.arctan2(y2 - y1, x2 - x1) * (180 / np.pi)
            slopes.append(slope)
    # Фильтруем вертикальные линии, которые имеют угол ~90 или ~-90 градусов
    slopes = [slope for slope in slopes if not np.isclose(abs(slope), 90, atol=10)]
    return np.mean(slopes) if len(slopes) > 0 else 0

In [6]:
def rotate_image(image_path, rotate_folder):

    # Создание папки, если она не существует
    if not os.path.exists(rotate_folder):
        os.makedirs(rotate_folder)
    # Загрузка изображения
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Обнаружение краев на изображении
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # Обнаружение линий на изображении
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)

    # Вычисление среднего угла наклона линий
    angle = average_slope(lines)

    # Получение размеров изображения
    (h, w) = img.shape[:2]

    # Вычисление центра изображения
    center = (w // 2, h // 2)

    # Применение поворота
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    # Формирование уникального имени файла для каждого фрагмента
    output_path = os.path.join(rotate_folder, f'rotated_{os.path.splitext(os.path.basename(image_path))[0]}.jpg')

    # Сохранение результата
    cv2.imwrite(output_path, rotated)


#### Регулярные выражения для валидации полей

In [7]:
import re

# Функция нормализации строки для поиска ключей в анкете
def normalize_string(string):
    # Удаляем все неалфавитные символы и приводим строку к нижнему регистру
    return re.sub(r'[^a-zA-Zа-яА-Я0-9]', '', string).lower()

clean_string = normalize_string("Строка! с разными# символами& и числами 123.")
print(clean_string)


строкасразнымисимволамиичислами123


#### Функции Разбор списка в финальный датафрейм по привязке к ключевому слову

In [8]:
# Функция для разделения списка словарей на отдельные словари анкет по ключевому слову
def split_on_keyword(data, keyword):
    result = []
    current_list = []

    for item in data:
        if item['text'].lower().strip() == keyword:
            if current_list:
                result.append(current_list)
            current_list = [item]
        else:
            current_list.append(item)
    
    # Добавляем последний список, если он не пустой
    if current_list:
        result.append(current_list)

    return result

In [9]:
# Функция объединения текстовых полей и поиска уникальных ключей, содержащихся в распозноваемой анкете
def merge_texts(sublist, field_list):
    new_list = []
    miss_next = False
    found_next = False
    for i in range(0, len(sublist) - 1):
        if miss_next:
            miss_next = False
            continue
        found = False
        merged_text = sublist[i]['text'] + ' ' + sublist[i + 1]['text']
        for el in field_list:
            if normalize_string(merged_text) in normalize_string(el) or\
                levenshtein_distance(normalize_string(merged_text),normalize_string(el))<=1:
            
                new_dict = {
                    'text': merged_text,
                    'x_coord': sublist[i + 1]['x_coord'],
                    'y_coord': sublist[i + 1]['y_coord'],
                    'width': sublist[i + 1]['width'],
                    'height': sublist[i + 1]['height']
                }
                new_list.append(new_dict)
                found = True
                found_next = True
                miss_next = True
                break
            
        if not found:
            new_list.append(sublist[i])
    
    # Добавляем последний элемент, если он не был объединен
    if not miss_next and len(sublist) > 0:
        new_list.append(sublist[-1])
    
    # Если были найдены объединения, рекурсивно вызываем функцию с новым списком
    if found_next:
        return merge_texts(new_list, field_list)
    else:
        return new_list

In [10]:
# Функция получения ключей с координатами для дальнейшего разбора и поиска распознанного рукописного текста
def process_text_fields(spisok, field_list):
    ext_list = {}
    for item in spisok:
        for field in field_list:
            if levenshtein_distance(normalize_string(item['text']),normalize_string(field))<=1 or\
                normalize_string(item['text']) == normalize_string(field) and len(item['text']) >= 3:
            
                if field in ext_list:
                    continue
                else:
                    ext_list[field] = {
                                    'x_coord': item['x_coord'],
                                    'y_coord': item['y_coord'],
                                    'width': item['width'],
                                    'height': item['height']}
                    break

    # Находим окончание поля для рукописного текста
    for key in ext_list: 
        found = False
        for other_key in ext_list:
            if key != other_key:
                if abs(int(ext_list[key]['y_coord'][1])/int(ext_list[key]['height'])*100 - int(ext_list[other_key]['y_coord'][1])/int(ext_list[other_key]['height'])*100) <= 1.2 and\
                int(ext_list[key]['x_coord'][1])< int(ext_list[other_key]['x_coord'][1]):
                    ext_list[key]['end_segment'] = ext_list[other_key]['x_coord'][1]
                    found = True
                    break
        if not found:
            ext_list[key]['end_segment'] = ext_list[key]['width']
    
    # после того, как все ключи найдены и записаны в ext_list, формируем очищенный список значений filtred_spisok для дальнейшего разбора
    filtered_spisok = []

    for i in spisok:
        # Устанавливаем флаг, указывающий на необходимость удаления элемента
        remove_item = False
        # Перебираем ключи в ext_list
        for key in ext_list:
            # Проверяем условие, если текст элемента i входит в ключ key
            if normalize_string(i['text']) in normalize_string(key) and len(i['text']) >= 3:
                remove_item = True
                break  # Если условие выполнено, выходим из внутреннего цикла
        # Если флаг не установлен, добавляем элемент в новый список
        if not remove_item:
            filtered_spisok.append(i)
    return ext_list, filtered_spisok

In [30]:
# Функция обработки распознанного текста и запись его в строку
def process_extract_text(text_fields, extracted_data):
    for key in extracted_data:
        proc_string = ''
        for item in text_fields:
            if abs(int(item['y_coord'][1])/int(item['height'])*100 - int(extracted_data[key]['y_coord'][1])/int(extracted_data[key]['height'])*100) <=1.2 and\
            int(extracted_data[key]['x_coord'][2]) <= int(item['x_coord'][1]) < int(extracted_data[key]['end_segment'])-5:
                proc_string += item['text']+' ' 
        extracted_data[key]= proc_string

#### Распознавание чекбоксов и запись текста 

In [12]:
# Функция поиска чек боксов и разделения их на заполненные и незаполненные
def checkbox_detect(image_path):
    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    cfg = config.PipelinesConfig()

    cfg.width_range = (25,87)
    cfg.height_range = (25,87)

    cfg.scaling_factors = [1.3]

    cfg.wh_ratio_range = (0.8, 1.2)


    cfg.group_size_range = (1, 1)

    cfg.dilation_iterations = 0

    checkboxes = get_checkboxes(
        image_path, cfg=cfg, px_threshold=0.2, plot=False, verbose=False)
    
    # определение конца сегмента для распознанного текста путем сравнения координат
    # чек боксов, стоящих на одной линии
    checkbox_coord = []

    for el in checkboxes:
        found = False
        for i in checkboxes:
            if el[0] != i[0]:
                if abs(int(el[0][1])/int(height)*100-int(i[0][1])/int(height)*100)<=1 and\
                int(el[0][0]) < int(i[0][0]):
                    coord = el[0] +(int(i[0][0]), int(height))
                    el = np.array([coord, el[1], el[2]], dtype=object)
                    checkbox_coord.append(el)
                    found = True
                    break
        if not found:
            coord = el[0] +(int(width), int(height))
            el = np.array([coord, el[1], el[2]], dtype=object)
            checkbox_coord.append(el)
                    
    # Разделение списков чек боксов на заполненные и незаполненные
    boxes = []
    checked_box = []

    for checkbox in checkbox_coord:
        boxes.append(checkbox[0])
        
    found = False
    for checkbox in checkbox_coord:
        if checkbox[1]:
            checked_box.append(checkbox[0])
            found = True
    if not found:
       checked_box.append((1,1,1,1,1,1))
        
    return boxes, checked_box
        

In [13]:
# Функция разделения списков заполненных чекбоксов на несколько
# в случае если на листе несколько анкет
def split_checkbox_list(data, checkbox_list, keyword):
    # Создаем список ключевых слов с координатами
    keywords = []

    for item in data:
        if item['text'].lower().strip() == keyword.lower().strip():
            keywords.append(item)

    # Проверяем, есть ли ключевые слова
    if not keywords:
        return []

    # Сортируем ключевые слова по координате Y начала
    sorted_keywords = sorted(keywords, key=lambda k: int(k['y_coord'][0]))

    # Если ключевое слово одно, создаем один диапазон до конца страницы
    if len(sorted_keywords) == 1:
        ranges = [(int(sorted_keywords[0]['y_coord'][1]), None)]
    else:
        # Получаем диапазоны между ключевыми словами
        ranges = [(int(sorted_keywords[i]['y_coord'][1]), int(sorted_keywords[i+1]['y_coord'][0]))
                  for i in range(len(sorted_keywords) - 1)]
        # Добавляем диапазон для последнего ключевого слова
        last_keyword = sorted_keywords[-1]
        ranges.append((int(last_keyword['y_coord'][1]), None))

    # Разделяем чекбоксы по диапазонам
    split_chbox_lists = [[] for _ in range(len(ranges))]
    
    for checkbox in checkbox_list:
        y_coord = checkbox[1]  # Y координата верхнего левого угла чекбокса
        
        for i, (start, end) in enumerate(ranges):
            if end is None:
                # Если это последний диапазон
                if y_coord > start:
                    split_chbox_lists[i].append(checkbox)
            else:
                if start < y_coord <= end:
                    split_chbox_lists[i].append(checkbox)
                    found = True
                    break
            

    return split_chbox_lists


In [14]:
# Функция заполнения слов по отмеченным чек-боксам
def check_box_extract_text(text_fields, checked_box):
    interest = []
    for el in checked_box:
        proc_string = ''
        for item in text_fields:
            # Прямое сравнение координат без индексации строки
            if abs(el[1]/el[5]*100 - int(item['y_coord'][0])/int(item['height'])*100) <= 1.2 and\
               el[0] < int(item['x_coord'][1]) < el[4]:
                proc_string += item['text'] + ' '
        # Добавляем proc_string в interest, если он не пустой
        if proc_string:
            interest.append(proc_string.rstrip(' '))
    return interest



#### Загрузка и предобработка

In [127]:
# Пример использования функции
clear_folder('proc/cropped')
clear_folder('proc/binary')
clear_folder('proc/rotated')
clear_folder('proc/output_images')
clear_folder('proc/norm_brig')
clear_folder('proc/bal_white')

In [128]:
# Путь к рабочему файлу
file = 'Ank_chbox.pdf'
file_path = f"input/input_pdf/{file}"
# берем файл разбираем его на изображения
file_to_png(file_path, 'proc/output_images')

Done!


In [129]:
rotate_folder = "proc/rotated"
for img in os.listdir('proc/output_images'):
    image_path = os.path.join('proc/output_images', img)

    rotate_image(image_path, rotate_folder)

#### Отправка запроса в Yandex OCR

In [130]:


result = []
for img in os.listdir('proc/rotated'):
    image_path = os.path.join('proc/rotated', img)
    
    # Заголовки запроса
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {IAM_TOKEN}',
        'x-folder-id': folder_id,
        'x-data-logging-enabled': 'true'
    }

    body = {
        "mimeType": "image",
        "languageCodes": ["ru", "en"],  
        "model": "handwritten",
        "content": encode_file(image_path)
    }
   
    # Отправка POST-запроса
    response = requests.post(VISION_URL, headers=headers, json=body)
    result.append(response)

In [131]:
for res in result:
    print(res.text)

{"result":{"textAnnotation":{"width":"1652","height":"2323","blocks":[{"boundingBox":{"vertices":[{"x":"665","y":"160"},{"x":"665","y":"191"},{"x":"853","y":"191"},{"x":"853","y":"160"}]},"lines":[{"boundingBox":{"vertices":[{"x":"665","y":"160"},{"x":"665","y":"191"},{"x":"853","y":"191"},{"x":"853","y":"160"}]},"text":"АНКЕТА","words":[{"boundingBox":{"vertices":[{"x":"665","y":"155"},{"x":"665","y":"197"},{"x":"853","y":"197"},{"x":"853","y":"155"}]},"text":"АНКЕТА","entityIndex":"-1","textSegments":[{"startIndex":"0","length":"6"}]}],"textSegments":[{"startIndex":"0","length":"6"}],"orientation":"ANGLE_0"}],"languages":[{"languageCode":"ru"}],"textSegments":[{"startIndex":"0","length":"6"}]},{"boundingBox":{"vertices":[{"x":"36","y":"236"},{"x":"36","y":"289"},{"x":"197","y":"289"},{"x":"197","y":"236"}]},"lines":[{"boundingBox":{"vertices":[{"x":"36","y":"236"},{"x":"36","y":"289"},{"x":"197","y":"289"},{"x":"197","y":"236"}]},"text":"Фамилия","words":[{"boundingBox":{"vertices":[

In [ ]:
data = json.loads(result[0].content.decode('UTF-8'))
data

#### Разбор полученного Respons в рабочий список

In [100]:
# extract lines of text
proc_list = []

for res in result:
    data = json.loads(res.content.decode('UTF-8'))
    for block in data["result"]["textAnnotation"]["blocks"]:
        width = data["result"]["textAnnotation"]["width"]  # Получаем ширину
        height = data["result"]["textAnnotation"]["height"]  # Получаем высоту
        for line in block['lines']:
            text = line['text']
            x_coord = [(vertex['x']) for vertex in line['boundingBox']['vertices']]
            y_coord = [(vertex['y']) for vertex in line['boundingBox']['vertices']]
            proc_list.append([text, x_coord, y_coord, width, height])
    

df= pd.DataFrame(proc_list)
df.head()
df.to_excel('test_lines.xlsx', index=False)    

In [36]:
# Для просмотра в Excel extract words of text
proc_list = []

for res in result:
    data = json.loads(res.content.decode('UTF-8'))
    for block in data["result"]["textAnnotation"]["blocks"]:
        width = data["result"]["textAnnotation"]["width"]  # Получаем ширину
        height = data["result"]["textAnnotation"]["height"]  # Получаем высоту
        for line in block['lines']:
            for word in line['words']:
                text = word['text']
                x_coord = [(vertex['x']) for vertex in word['boundingBox']['vertices']]
                y_coord = [(vertex['y']) for vertex in word['boundingBox']['vertices']]
                proc_list.append([text, x_coord, y_coord,  width, height])

df= pd.DataFrame(proc_list)
df.head()
df.to_excel('test.xlsx', index=False)    

In [132]:
# для основного кода extract words of text кортежи
proc_list = []

for res in result:
    data = json.loads(res.content.decode('UTF-8'))
    elenco= []
    for block in data["result"]["textAnnotation"]["blocks"]:
        width = data["result"]["textAnnotation"]["width"]  # Получаем ширину
        height = data["result"]["textAnnotation"]["height"]  # Получаем высоту
        for line in block['lines']:
            #x_coord_line = [(vertex['x']) for vertex in line['boundingBox']['vertices']]
            #y_coord_line = [(vertex['y']) for vertex in line['boundingBox']['vertices']]
            for word in line['words']:
                text = word['text']
                x_coord = [(vertex['x']) for vertex in word['boundingBox']['vertices']]
                y_coord = [(vertex['y']) for vertex in word['boundingBox']['vertices']]
                elenco.append({"text": text, "x_coord": x_coord, "y_coord": y_coord, "width": width, "height": height})
    proc_list.append(elenco)

In [141]:
proc_list[3]

[{'text': 'АНКЕТА',
  'x_coord': ['684', '684', '853', '853'],
  'y_coord': ['156', '198', '198', '156'],
  'width': '1661',
  'height': '2343'},
 {'text': 'Фамилия',
  'x_coord': ['37', '37', '191', '191'],
  'y_coord': ['252', '293', '293', '252'],
  'width': '1661',
  'height': '2343'},
 {'text': 'Романов',
  'x_coord': ['299', '299', '526', '526'],
  'y_coord': ['256', '298', '298', '256'],
  'width': '1661',
  'height': '2343'},
 {'text': 'Имя',
  'x_coord': ['1015', '1015', '1100', '1100'],
  'y_coord': ['250', '285', '285', '250'],
  'width': '1661',
  'height': '2343'},
 {'text': 'Аитом',
  'x_coord': ['1160', '1160', '1337', '1337'],
  'y_coord': ['252', '289', '289', '252'],
  'width': '1661',
  'height': '2343'},
 {'text': 'Отчество',
  'x_coord': ['44', '44', '158', '158'],
  'y_coord': ['342', '367', '367', '342'],
  'width': '1661',
  'height': '2343'},
 {'text': 'валерьевич',
  'x_coord': ['250', '250', '550', '550'],
  'y_coord': ['338', '388', '388', '338'],
  'width':

### Поиск чекбоксов

In [133]:
boxes_list = []
checkbox_list = []
for img in os.listdir('proc/rotated'):
    image_path = os.path.join('proc/rotated', img)  
    boxes, checked_box = checkbox_detect(image_path)
    boxes_list.append(boxes)
    checkbox_list.append(checked_box)

Processing file:  proc/rotated\rotated_img1.jpg
Processing file:  proc/rotated\rotated_img2.jpg
Processing file:  proc/rotated\rotated_img3.jpg
Processing file:  proc/rotated\rotated_img4.jpg
Processing file:  proc/rotated\rotated_img5.jpg


#### Получение результата

In [134]:
res_list = []
field_list = [
    'Фамилия',
    'ФИО',
    'ФИО родителя',
    'Имя',
    'Отчество',
    'Дата Рождения',
    'E-mail',
    'Телефон',
    'Телефон родителя',
    'Мобильный телефон',
    'Город',
    'Хобби',
    'Регион',
    'Населенный пункт',
    'Адрес',
    'Паспорт',
    'Дата выдачи',
    'Название учебного заведения',
    'Школа/Колледж',
    'Курс\класс'
]

In [135]:
sublist = []
for el in proc_list:
    split_lists = split_on_keyword(el, 'анкета')
    for item in split_lists:
        sublist.append(item)
        
for spisok in sublist:
    new_sublist = merge_texts(spisok,field_list)
    extracted_data, filtered_spisok = process_text_fields(new_sublist, field_list)
    process_extract_text(filtered_spisok, extracted_data)
    res_list.append(extracted_data)

splited_checkboxes = []
for el, i in zip(proc_list, checkbox_list):
    split_cb = split_checkbox_list(el, i, 'анкета')
    for item in split_cb:
        splited_checkboxes.append(item)

interest_list =[]
for res, cb in zip(sublist, splited_checkboxes):
    interest_list.append(check_box_extract_text(res, cb))


In [117]:
#def validate_phone(phone_str):
    #return re.sub(r'^(\+?\d+(?:[\d\s]*\d)?)(?:\s+[А-Яа-яA-Za-z].*)?$', r'\1', phone_str)




In [136]:
def validate_phone(phone_str):
    # Проверка на NaN значения
    if pd.isna(phone_str):
        return None  # Если значение NaN, возвращаем None
    # Сначала проверяем, начинается ли значение с буквы
    if re.match(r'^[А-Яа-яA-Za-z]', phone_str):
        return None  # Если начинается с буквы, возвращаем None
    # Далее идет уже существующая логика поиска и валидации номера телефона
    return re.sub(r'^(\+?\d+(?:[\d\s]*\d)?)(?:\s+[А-Яа-яA-Za-z].*)?$', r'\1', phone_str)

In [138]:
def create_lead_title(entry):
    # Возвращаем ФИО, если оно есть
    if 'ФИО' in entry:  
        return entry['ФИО'].strip()
    # Иначе обрабатываем составные части
    else:
        parts = [entry.get(part, '').strip() for part in ['Фамилия', 'Имя', 'Отчество']]
        # Объединяем части, исключая пустые строки
        result = ' '.join(part for part in parts if part)
        return result

In [139]:
df_res = pd.DataFrame(res_list)
df_res['Название лида'] = [create_lead_title(item) for item in res_list]
df_res['E-mail'] = df_res['E-mail'].map(lambda x: x.replace(' ', '') if isinstance(x, str) else x)
df_res['Дата Рождения'] = df_res['Дата Рождения'].map(lambda x: x.replace(' ', '') if isinstance(x, str) else x)
df_res['Телефон']   =  df_res['Телефон'].apply(validate_phone).map(lambda x: x.replace(' ', '') if isinstance(x, str) else x)
df_res['Телефон родителя']   =  df_res['Телефон родителя'].apply(validate_phone).map(lambda x: x.replace(' ', '') if isinstance(x, str) else x) 
df_res.dropna(how='all', inplace=True)
df_res['интерес'] = interest_list
df_res.to_excel(f'test_result/test_{file}.xlsx', index=False)
df_res

,Фамилия,Имя,Отчество,Дата Рождения,Телефон,Телефон родителя,E-mail,Хобби,Регион,Населенный пункт,Адрес,Паспорт,Дата выдачи,Название учебного заведения,Курс\класс,ФИО,Название лида,интерес
0,Cshogol,Роман,Дмитриевич,27.10.1998,,,,,Московская область,,Кирова 47,4621 553498,10.03 2022,Мецей,,NaN,Cshogol Роман Дмитриевич,[]
1,Кавалёв,Андрей,Николаевич,,89774215318,,KovalerYander.ru,,Москва,Москва,жулебенския б- 13,4513 491825,08.03. 2020,,,NaN,Кавалёв Андрей Николаевич,[]
2,Попов,Иван,Ибратимович,04.11.1991,+7-910-222-12-39,+7(910)888-66-44,bestrom.ru,работа,,Москва,Основной д.1,14.20.11 22 33,,Чкивер №228,11 (выпускной),NaN,Попов Иван Ибратимович,"[Программирование, Управлениение, Экономика]"
3,Иван,,Иван,01.01.2000,,89103321133,pmailcom,,,Можва,"Месковская,",14.22 33 44.66,01.01. 1999,,NaN,NaN,Иван Иван,"[Управлениение, Психология]"
4,,Синица,,08.05.2000,(917)G543210,7/489/5643,,,,Роще,Пр- Лепины.,1234 NS67890,23. 04. 2020,,,NaN,Синица,[]
5,Лебедь,Олег,,11.05.67,,46371,poshtragmail.Lon,стрельба,,Москва,Спаванский бультар. кв,48 78 5321ts,02. 02. 2005,,,NaN,Лебедь Олег,[]
6,Романов,Аитом,валерьевич,03.09.1922,+79535693185,+79104021415,romanov@ramblerru,велосипед,Москва,Москве,ул неглинная кв.,631729,10.05. 2014,школа ----------------------------------------...,,NaN,Романов Аитом валерьевич,[Экономика Психология]
7,,Александра,Олеговна,NaN,99955503.21,+79031587611,(sXXXbrothers.com,гимнастика,американский пункт Чиладель,NaN,Красных фонефей дв кв5,3234 5467525,19.05. 1999,институт благородных 12,NaN,,,[]
8,Шреков,Щрек,Оркович,01.01.1900,35344367254496728,None,shreckboloto.com,купание,тредисятое цирство,Болато,3-е болото строителей д.20,оркам писпорт не дакт,нет.,не учился,,NaN,Шреков Щрек Оркович,[]
9,Васечкин.,,кергеевич,21.06.1980,,9152572255,adventure@vandex.ru,проделкл,России,Навороссий сы,Мартовск ул д10 173,3728 161467,25. 06 1396,NaN,,NaN,Васечкин. кергеевич,"[Робототехника, Психология]"
